In [1]:
#%matplotlib inline
import matplotlib.pylab as plt

import numpy as np
from numpy.linalg import inv
import pandas as pd
from pandas_datareader import data as web

from scipy.optimize import minimize

In [2]:
# 무위험수익률, 수익률,공분산으로
# 샤프비율을 최대로 하는 접점포트폴리오 비중 계산 
def solveWeights(R, C, rf):
    
    # 파이썬은 함수안에 함수를 정의할 수 있다
    # 최적비중계산을 위해 다음과 같이 목적함수를 정의한다
    def obj(W, R, C, rf):
        mean = sum(R * W)
        var = np.dot(np.dot(W, C), W)
        #샤프비율을 효용함수로 한다
        util = (mean - rf) / np.sqrt(var) 
        # 효용함수 극대화= 효용함수 역함수를 최소화하는 것이다.
        return 1 / util  

    n = len(R)  # 투자자산 갯수

    # 동일비중으로 최적화 시작
    W = np.ones([n]) / n  
    # 비중범위는 0~100%사이(공매도나 차입조건이 없음)
    bnds = [(0., 1.) for i in range(n)]  
    # 제약조건은 비중합=100%
    cons = ({'type': 'eq', 'fun': lambda W: sum(W) - 1.}) 
    # 최적화
    res = minimize(obj, W, (R, C, rf), method='SLSQP', constraints=cons, bounds=bnds)
    if not res.success:
      # 최적화 실패한 경우 
      raise BaseException(res.message)
    # 최적화 결과를 돌려준다  
    return res.x

In [3]:
# 무위험수익률, 수익률,공분산으로 효율적 프런티어 계산
def solveFrontier(R, C, rf):

    # 파이썬은 함수안에 함수를 정의할 수 있다
    # 최적비중계산을 위해 다음과 같이 목적함수를 정의한다
    def obj(W, R, C, r):
        # 주어진 수익률에서 분산을 최소로 하는 비중 계산
        mean = sum(R * W)
        var = np.dot(np.dot(W, C), W)
        # 최적화 제약조건 페널티
        penalty = 100 * abs(mean - r)             
        return var + penalty

    # 효율적 프론티어를 구성하는 평균-분산을 돌려줄
    # 리스트를 미리 준비한다
    frontier_mean, frontier_var = [], []

    n = len(R)  # 투자자산 갯수

    # 수익률 최저~최대 사이를 반복한다
    for r in np.linspace(min(R), max(R), num=20):
        # 최적화 함수에 전달할 초기값으로 동일비중으로 시작한다
        W = np.ones([n]) / n 

        # 최적화 함수에 전달할 범위조건과 제약조건을 미리 준비한다
        # 범위조건: 각 구성자산의 투자비중은 0~100% 사이이며
        # 제약조건: 전체 투자비중은 100%이다.
        bnds = [(0, 1) for i in range(n)]
        cons = ({'type': 'eq', 'fun': lambda W: sum(W) - 1.})
        
        # 최적화 함수 minimize()은 최적화할 obj함수와 
        # 최적화를 시작할 초깃값을 인수로 받는다.         
        res = minimize(obj, W, (R, C, r), method='SLSQP', constraints=cons, bounds=bnds)
        if not res.success:
            # 최적화에 실패한 경우
            raise BaseException(res.message)
        
        # 효율적 프런티어 평균과 분산리스트에 
        # 최적포트폴리오 수익률과 분산 추가
        frontier_mean.append(r)
        frontier_var.append(np.dot(np.dot(res.x, C), res.x))
    return np.array(frontier_mean), np.array(frontier_var)

In [4]:
# 효율적 포트폴리오 최적화        
def optimize_frontier(R, C, rf):

    # 접점포트폴리오 계산
    W = solveWeights(R, C, rf)
    # 투자비중으로 계산한 평균과 분산
    tan_mean = sum(R * W)
    tan_var = np.dot(np.dot(W, C), W)    

    # 효율적 포트폴리오 계산
    eff_mean, eff_var = solveFrontier(R, C, rf)

    # 비중, 접점포트폴리오의 평균/분산, 효율적 포트폴리오의 평균/분산을
    # 딕셔너리 데이터형으로 돌려준다    
    return {'weights':W, 'tan_mean':tan_mean, 'tan_var':tan_var, 'eff_mean':eff_mean, 'eff_var':eff_var}

In [5]:
# 자산에 대한 투자자의 전망과 전망의 기대수익률을 행렬로 만든다
def CreateMatrixPQ(names, views):
    r, c = len(views), len(names)

    # 투자전망과 기대수익률 행렬
    # views[i][3]에는 기대수익률을 가리킴
    Q = [views[i][3] for i in range(r)]     

    # 전망행렬 P를 만들기 위해
    # 구성자산 딕셔너리 작성
    nameToIndex = dict()    
    for i, n in enumerate(names):
        nameToIndex[n] = i

    # 투자전망 
    P = np.zeros([r, c]) 
    for i, v in enumerate(views):
        # 가령 전망이 ('MSFT', '>', 'GE', 0.02) 이라면
        # views[i][0] <-- 'MSFT' --> name1
        # views[i][1] <-- '>'
        # views[i][2] <-- 'GE'   --> name2
        # views[i][3] <-- '0.02'
        name1, name2 = views[i][0], views[i][2]
        P[i, nameToIndex[name1]] = +1 if views[i][1] == '>' else -1
        P[i, nameToIndex[name2]] = -1 if views[i][1] == '>' else +1
    return np.array(Q), P

In [6]:
tickers=['PFE','INTC','NFLX','JPM','XOM','GOOG','JNJ','AAPL','AMZN']
cap = {'PFE':201102000000,'INTC':257259000000,'NFLX':184922000000,
       'JPM':272178000000,'XOM':178228000000,'GOOG':866683000000,
       'JNJ':403335000000,'AAPL':1208000000000,'AMZN':1178000000000
      }
prices, caps = [], []

for s in tickers:    
    pxclose = web.DataReader(s, data_source='yahoo',start='01-01-2018', end='31-12-2019')['Adj Close']
    prices.append(list(pxclose))  
    caps.append(cap[s])

In [7]:
n = len(tickers)
W = np.array(caps) / sum(caps) # 시가총액의 비율계산
prices = np.matrix(prices)  # prices를 numpy matrix로 변환

# 수익률 행렬을 만들어 계산
rows, cols = prices.shape
returns = np.empty([rows, cols - 1])
for r in range(rows):
    for c in range(cols - 1):
        p0, p1 = prices[r, c], prices[r, c + 1]
        returns[r, c] = (p1 / p0) - 1

# 수익률계산
expreturns = np.array([])
for r in range(rows):
    expreturns = np.append(expreturns, np.mean(returns[r]))

# 공분산계산
covars = np.cov(returns)
R = (1 + expreturns) ** 250 - 1  # 연율화
C = covars * 250  # 연율화

# 무위험 이자율
rf = .015 

In [8]:
expreturns

array([ 0.00036199,  0.00077312,  0.00127814,  0.00069936, -0.00013888,
        0.00058917,  0.00027701,  0.00127331,  0.00105956])

In [9]:
display(pd.DataFrame({'Return': R, 'Weight (based on market cap)': W}, index=tickers).T)

,PFE,INTC,NFLX,JPM,XOM,GOOG,JNJ,AAPL,AMZN
Return,0.094701,0.213133,0.376207,0.190983,-0.034127,0.158643,0.071697,0.374548,0.303104
Weight (based on market cap),0.042340,0.054163,0.038933,0.057304,0.037524,0.182471,0.084918,0.254331,0.248015


In [10]:
display(pd.DataFrame(C, columns=tickers, index=tickers))

,PFE,INTC,NFLX,JPM,XOM,GOOG,JNJ,AAPL,AMZN
PFE,0.037363,0.021650,0.026020,0.015811,0.015772,0.018532,0.018269,0.019717,0.021559
INTC,0.021650,0.093410,0.054385,0.029809,0.027001,0.039814,0.017512,0.045336,0.042400
NFLX,0.026020,0.054385,0.165254,0.031729,0.027571,0.060201,0.020956,0.053069,0.081353
JPM,0.015811,0.029809,0.031729,0.041792,0.022180,0.028015,0.014417,0.027610,0.029125
XOM,0.015772,0.027001,0.027571,0.022180,0.040334,0.024196,0.014377,0.023426,0.022529
GOOG,0.018532,0.039814,0.060201,0.028015,0.024196,0.068071,0.016314,0.044830,0.054362
JNJ,0.018269,0.017512,0.020956,0.014417,0.014377,0.016314,0.037225,0.015872,0.016410
AAPL,0.019717,0.045336,0.053069,0.027610,0.023426,0.044830,0.015872,0.075180,0.051311
AMZN,0.021559,0.042400,0.081353,0.029125,0.022529,0.054362,0.016410,0.051311,0.090502
